In [49]:
import geopandas as gpd
import pandas as pd
import numpy as np

# User group 3

In [50]:
#read in the cleaned pedestrian data
pedestrian_network = gpd.read_file('../data/cleaned-pednetwork/cleaned-pednetwork-full/cleaned-pednetwork-full.shp')

# Sidewalks

## 1. Waytype

In [51]:
pedestrian_network['fclass'].unique()  

array(['crossing', 'footway', 'steps', 'pedestrian', 'path', 'track',
       'service', 'm_footway', 'residential', 'm_service', 'cycleway',
       'tertiary', 'secondary', 'trunk', 'primary', '0'], dtype=object)

In [52]:
#create a mapping of the fclass to waytype weight
waytype_weight_dict = {
    'service': 2,
    'm_service': 2,
    'residential': 2,
    'tertiary':4,
    'secondary': 4,
    'primary': 4,
    'trunk': 4,
    'cycleway': 2,
}


In [53]:
pedestrian_network['waytype_w'] = pedestrian_network['fclass'].map(waytype_weight_dict).fillna(1).astype(float)

## 2. Sidewalk width

In [54]:
# read in the sidewalk width data
sidewalk_width = pd.read_csv('../data/pednetwork-attributes/sidewalks/paving-and-width.csv')

In [55]:
pedestrian_network

,pednet_id,pednet_pid,osm_id,type,fclass,crossing_t,z-start,z-end,geometry,waytype_w
0,0.0,0,1.000034e+09,ped,crossing,m,15.400000,16.200001,"LINESTRING (1277935.51 238384.953, 1277942.83 ...",1.0
1,1.0,1,1.000034e+09,ped,footway,0,16.200001,16.200001,"LINESTRING (1277978.83 238310.174, 1277966.394...",1.0
2,2.0,2,1.000034e+09,ped,footway,0,10.500000,16.200001,"LINESTRING (1278374.334 238398.46, 1278317.326...",1.0
3,3.0,3,1.005452e+09,ped,steps,0,11.600000,11.600000,"LINESTRING (1284223.335 235156.549, 1284211.29...",1.0
4,4.0,4,1.005641e+09,ped,steps,0,124.599998,118.900002,"LINESTRING (1266761.001 234328.359, 1266840.43...",1.0
...,...,...,...,...,...,...,...,...,...,...
34928,32959.0,34924,1.124353e+09,ped,footway,0,68.199997,68.199997,"LINESTRING (1262631.523 239983.092, 1262641.40...",1.0
34929,32960.0,34925,1.700162e+08,road,service,0,65.099998,62.700001,"LINESTRING (1262862.932 240024.343, 1262868.78...",2.0
34930,32961.0,34926,1.700162e+08,road,service,0,62.700001,65.099998,"LINESTRING (1262918.344 240036.228, 1262921.4 ...",2.0
34931,32962.0,34927,6.423613e+06,road,service,0,61.599998,62.700001,"LINESTRING (1263190.244 239825.195, 1263057.2 ...",2.0


In [56]:
#join on pednet_id
pedestrian_network = pedestrian_network.merge(sidewalk_width, on='pednet_pid', how='left')

In [57]:
def calculate_width_w(width):
    if pd.isna(width):
        return 1
    elif width < 48:
        return 5
    elif 48 <= width < 72:
        return 2
    elif width >= 72 :
        return 1

In [58]:
pedestrian_network['width_w'] = pedestrian_network['width'].apply(calculate_width_w)


## 3. Sidewalk surface type

In [59]:
pedestrian_network['surface_t'].unique()

array(['concrete', 'gravel', 'asphalt', 'brick'], dtype=object)

In [60]:
#craete a mapping of the surface type to surface weight
surface_weight_dict = {
    'concrete': 1,
    'asphalt': 2,
    'brick': 3,
    'gravel': 5,
}

In [61]:
#map the surface type to the surface weight
pedestrian_network['surface_w'] = pedestrian_network['surface_t'].map(surface_weight_dict).fillna(1).astype(float)

## 4. Number of steps

In [62]:
# read in the number of steps data
number_of_steps = pd.read_csv('../data/pednetwork-attributes/sidewalks/number-of-steps.csv')

In [63]:
#join on pednet_id
pedestrian_network = pedestrian_network.merge(number_of_steps, on='pednet_id', how='left')

In [64]:
pedestrian_network

,pednet_id,pednet_pid,osm_id,type,fclass,crossing_t,z-start,z-end,geometry,waytype_w,surface_t,width,width_w,surface_w,number-of-steps
0,0.0,0,1.000034e+09,ped,crossing,m,15.400000,16.200001,"LINESTRING (1277935.51 238384.953, 1277942.83 ...",1.0,concrete,72.0,1,1.0,NaN
1,1.0,1,1.000034e+09,ped,footway,0,16.200001,16.200001,"LINESTRING (1277978.83 238310.174, 1277966.394...",1.0,concrete,106.0,1,1.0,NaN
2,2.0,2,1.000034e+09,ped,footway,0,10.500000,16.200001,"LINESTRING (1278374.334 238398.46, 1278317.326...",1.0,concrete,61.0,2,1.0,NaN
3,3.0,3,1.005452e+09,ped,steps,0,11.600000,11.600000,"LINESTRING (1284223.335 235156.549, 1284211.29...",1.0,concrete,72.0,1,1.0,1.0
4,4.0,4,1.005641e+09,ped,steps,0,124.599998,118.900002,"LINESTRING (1266761.001 234328.359, 1266840.43...",1.0,concrete,72.0,1,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34928,32959.0,34924,1.124353e+09,ped,footway,0,68.199997,68.199997,"LINESTRING (1262631.523 239983.092, 1262641.40...",1.0,concrete,72.0,1,1.0,NaN
34929,32960.0,34925,1.700162e+08,road,service,0,65.099998,62.700001,"LINESTRING (1262862.932 240024.343, 1262868.78...",2.0,concrete,72.0,1,1.0,NaN
34930,32961.0,34926,1.700162e+08,road,service,0,62.700001,65.099998,"LINESTRING (1262918.344 240036.228, 1262921.4 ...",2.0,concrete,72.0,1,1.0,NaN
34931,32962.0,34927,6.423613e+06,road,service,0,61.599998,62.700001,"LINESTRING (1263190.244 239825.195, 1263057.2 ...",2.0,concrete,72.0,1,1.0,NaN


In [65]:
def calculate_steps_w(steps):
    if pd.isna(steps):
        return 1
    elif steps >= 1:
        return 999

In [66]:
#want to convert to steps_w Nan is 1, < 5 is 2, >=5 < 10 is 3, >=10 is 5
pedestrian_network['steps_w'] = pedestrian_network['number-of-steps'].apply(calculate_steps_w).astype(float)

In [67]:
# pedestrian_network

## 5. Slope

In [68]:
# create a new column for the length of the pedestrian network
pedestrian_network['length'] = pedestrian_network['geometry'].length

In [69]:
# add a column for z difference
pedestrian_network['z_d'] = abs(pedestrian_network['z-start'] - pedestrian_network['z-end'])
#change the z_d to meters to feet
pedestrian_network['z_d'] = pedestrian_network['z_d'] * 3.28084

In [70]:
# create a new column for the length of the pedestrian network
pedestrian_network['length'] = pedestrian_network['geometry'].length

In [71]:
#calculate the slope in percent
pedestrian_network['slope'] = pedestrian_network['z_d'] / pedestrian_network['length']

In [72]:
# add in a slope weight, if slope is between 0.05 and 0.1 then 5, if > 0.1 then 999, else 1
def calculate_slope_w(slope):
    if 0.05 <= slope < 0.1:
        return 5
    elif slope >= 0.1:
        return 999
    else:
        return 1

In [73]:
pedestrian_network['slope_w'] = pedestrian_network['slope'].apply(calculate_slope_w).astype(float)

# Crossings

In [74]:
#read in crossing traffic flow data
crossing_traffic_flow = pd.read_csv('../data/pednetwork-attributes/crosswalks/crossings-traffic-flow.csv')

In [75]:
# join on pednet_id
pedestrian_network = pedestrian_network.merge(crossing_traffic_flow, on='pednet_id', how='left')

In [76]:
pedestrian_network

,pednet_id,pednet_pid,osm_id,type,fclass,crossing_t,z-start,z-end,geometry,waytype_w,...,width,width_w,surface_w,number-of-steps,steps_w,length,z_d,slope,slope_w,traffic_flow
0,0.0,0,1.000034e+09,ped,crossing,m,15.400000,16.200001,"LINESTRING (1277935.51 238384.953, 1277942.83 ...",1.0,...,72.0,1,1.0,NaN,1.0,71.188703,2.624676,0.036869,1.0,low
1,1.0,1,1.000034e+09,ped,footway,0,16.200001,16.200001,"LINESTRING (1277978.83 238310.174, 1277966.394...",1.0,...,106.0,1,1.0,NaN,1.0,16.675574,0.000000,0.000000,1.0,NaN
2,2.0,2,1.000034e+09,ped,footway,0,10.500000,16.200001,"LINESTRING (1278374.334 238398.46, 1278317.326...",1.0,...,61.0,2,1.0,NaN,1.0,458.294269,18.700791,0.040805,1.0,NaN
3,3.0,3,1.005452e+09,ped,steps,0,11.600000,11.600000,"LINESTRING (1284223.335 235156.549, 1284211.29...",1.0,...,72.0,1,1.0,1.0,999.0,17.555658,0.000000,0.000000,1.0,NaN
4,4.0,4,1.005641e+09,ped,steps,0,124.599998,118.900002,"LINESTRING (1266761.001 234328.359, 1266840.43...",1.0,...,72.0,1,1.0,5.0,999.0,79.474146,18.700778,0.235306,999.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34928,32959.0,34924,1.124353e+09,ped,footway,0,68.199997,68.199997,"LINESTRING (1262631.523 239983.092, 1262641.40...",1.0,...,72.0,1,1.0,NaN,1.0,10.709117,0.000000,0.000000,1.0,NaN
34929,32960.0,34925,1.700162e+08,road,service,0,65.099998,62.700001,"LINESTRING (1262862.932 240024.343, 1262868.78...",2.0,...,72.0,1,1.0,NaN,1.0,60.364828,7.874008,0.130440,999.0,NaN
34930,32961.0,34926,1.700162e+08,road,service,0,62.700001,65.099998,"LINESTRING (1262918.344 240036.228, 1262921.4 ...",2.0,...,72.0,1,1.0,NaN,1.0,155.739448,7.874008,0.050559,5.0,NaN
34931,32962.0,34927,6.423613e+06,road,service,0,61.599998,62.700001,"LINESTRING (1263190.244 239825.195, 1263057.2 ...",2.0,...,72.0,1,1.0,NaN,1.0,344.187381,3.608932,0.010485,1.0,NaN


In [77]:
def calculate_crossing_w(row):
    if row['crossing_t'] == 'm':
        if row['traffic_flow'] == 'low':
            return 2
        elif row['traffic_flow'] == 'mid':
            return 3
        elif row['traffic_flow'] == 'high':
            return 5
    if row['crossing_t'] == 'u':
        if row['traffic_flow'] == 'low':
            return 3
        elif row['traffic_flow'] == 'mid':
            return 5
        elif row['traffic_flow'] == 'high':
            return 999
    return 1  

In [78]:
pedestrian_network['crossing_w'] = pedestrian_network.apply(calculate_crossing_w, axis=1).astype(float)

# Calculate length based on weight


In [79]:
#create a new column for the total weight that is the product of all the weights
pedestrian_network['total_w'] = pedestrian_network['waytype_w'] * pedestrian_network['width_w'] * pedestrian_network['surface_w'] * pedestrian_network['steps_w'] * pedestrian_network['slope_w'] * pedestrian_network['crossing_w']
# if the total weight is greater than 999, set it to 999
pedestrian_network['total_w'] = np.where(pedestrian_network['total_w'] > 999, 999, pedestrian_network['total_w'])

In [80]:
#create a column for ['weighted_length'] that length * total_w
pedestrian_network['w_length'] = pedestrian_network['length'] * pedestrian_network['total_w']

In [81]:
# pedestrian_network

In [82]:
# new column that calculates the ratio change of the weighted length
pedestrian_network['w_length_r'] = pedestrian_network['w_length']/pedestrian_network['length']

In [83]:
#new column that calculates the absolute difference between the weighted length and the length
pedestrian_network['w_length_d'] = pedestrian_network['w_length'] - pedestrian_network['length']

In [84]:
pedestrian_network.head()

,pednet_id,pednet_pid,osm_id,type,fclass,crossing_t,z-start,z-end,geometry,waytype_w,...,length,z_d,slope,slope_w,traffic_flow,crossing_w,total_w,w_length,w_length_r,w_length_d
0,0.0,0,1.000034e+09,ped,crossing,m,15.400000,16.200001,"LINESTRING (1277935.51 238384.953, 1277942.83 ...",1.0,...,71.188703,2.624676,0.036869,1.0,low,2.0,2.0,142.377406,2.0,71.188703
1,1.0,1,1.000034e+09,ped,footway,0,16.200001,16.200001,"LINESTRING (1277978.83 238310.174, 1277966.394...",1.0,...,16.675574,0.000000,0.000000,1.0,NaN,1.0,1.0,16.675574,1.0,0.000000
2,2.0,2,1.000034e+09,ped,footway,0,10.500000,16.200001,"LINESTRING (1278374.334 238398.46, 1278317.326...",1.0,...,458.294269,18.700791,0.040805,1.0,NaN,1.0,2.0,916.588537,2.0,458.294269
3,3.0,3,1.005452e+09,ped,steps,0,11.600000,11.600000,"LINESTRING (1284223.335 235156.549, 1284211.29...",1.0,...,17.555658,0.000000,0.000000,1.0,NaN,1.0,999.0,17538.102730,999.0,17520.547071
4,4.0,4,1.005641e+09,ped,steps,0,124.599998,118.900002,"LINESTRING (1266761.001 234328.359, 1266840.43...",1.0,...,79.474146,18.700778,0.235306,999.0,NaN,1.0,999.0,79394.672309,999.0,79315.198162


In [85]:
pedestrian_network = pedestrian_network[['pednet_id', 'waytype_w', 'width_w', 'surface_w', 'steps_w','slope_w', 'crossing_w', 'length', 'w_length', 'w_length_r', 'w_length_d', 'z-start' ,'z-end','geometry']]

Visual Checks

In [92]:
# pedestrian_network.explore(column = 'w_length_r', tiles = 'cartodbpositron', scheme = 'naturalbreaks')

In [87]:
# pedestrian_network['slope_w'].value_counts()

slope_w
1.0      23667
5.0       6688
999.0     4578
Name: count, dtype: int64

Saving Data

In [93]:
#save the data to shapefile
pedestrian_network.to_file('../data/weighted-network/user-group-3/user-group-3-weighted-network.shp')

In [89]:
# pedestrian_network

In [94]:
#save also as geojson
pedestrian_network.to_file('../data/paired-data/ug3/network_ug3.geojson', driver='GeoJSON')